In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from dropbot import SerialProxy
import pandas as pd

%matplotlib inline

def plot_and_fit(x, y):
    plt.plot(x, y, 'o')
    p = np.polyfit(x, y, 1)
    plt.plot(x, x * p[0] + p[1], 'k--')
    return p

try:
    proxy.terminate()
    del proxy
except:
    pass

# Connect to the connected DropBot and print it's properties
proxy = SerialProxy()

print proxy.properties

print "\nuuid\t\t\t%s\n" % str(proxy.uuid)

#proxy.reset_config()
proxy.update_config(max_voltage=140, i2c_address=31)

print "i2c_scan:", proxy.i2c_scan()
print "soft_i2c_scan:", proxy.soft_i2c_scan()

proxy.initialize_switching_boards()

print "number of channels:", proxy.number_of_channels

print "\nconfig:"
print proxy.config

In [ ]:
# test the measured voltage for a range of target voltages
proxy.voltage = proxy.min_waveform_voltage
proxy.hv_output_enabled = True
measured_voltage = []

voltage = np.linspace(proxy.min_waveform_voltage,
                      proxy.max_waveform_voltage,
                      10)[1:-1]
for v in voltage:
    proxy.voltage = v
    time.sleep(.5)
    measured_voltage.append(proxy.measure_voltage())

measured_voltage = np.array(measured_voltage)
proxy.voltage = proxy.min_waveform_voltage + 5

plt.figure()
# plot the measured vs target votage
plt.plot(voltage, measured_voltage, 'o')
plt.plot(voltage, voltage, 'k--')
plt.xlabel('Target voltage')
plt.ylabel('Measured voltage')

# calculate the average rms error
r = measured_voltage - voltage
print 'rms_error = %.1f%%' % (100 * np.sqrt(np.mean((r / voltage)**2)))

In [ ]:
# update the calibration values
R6 = 2e6
R7 = proxy.config.R7
pot_max = proxy.config.pot_max

R_pot = R6 / ( 2 * voltage / 1.5 - 1 ) - R7
x = R_pot / pot_max

plt.plot(x, 1 / (2 * np.array(measured_voltage) / 1.5 - 1), 'o')
p = np.polyfit(x, 1 / (2 * np.array(measured_voltage) / 1.5 - 1), 1)
print p
plt.plot(x, x*p[0] + p[1])
pot_max = p[0]*R6
R7 = np.abs(p[1]*R6)

print 'pot_max =', pot_max
print 'R7 =', R7
max_voltage = 1.5 / 2.0 * (R6 / R7 + 1)
min_voltage = 1.5 / 2.0 * (R6 / (pot_max + R7) + 1)
print 'min_voltage =', min_voltage 
print 'max_voltage =', max_voltage
proxy.update_config(pot_max=pot_max,
                    R7=R7,
                    #min_voltage=min_voltage,
                    #max_voltage=max_voltage
                   )

In [ ]:
# test how long it takes to change the actuation voltage
proxy.voltage = 40
time.sleep(5)
proxy.voltage = 100
t, v = zip(*[(time.time(), proxy.measure_voltage()) for x in range(10)])
t = np.array(t) - t[0]
plt.figure()
plt.plot(1e3*t, v, 'o')
plt.ylabel('voltage')
plt.xlabel('time (ms)')

proxy.voltage = 40 
t, v = zip(*[(time.time(), proxy.measure_voltage()) for x in range(100)])
t = np.array(t) - t[0]
plt.figure()
plt.plot(1e3*t, v, 'o')
plt.ylabel('voltage')
plt.xlabel('time (ms)')

In [ ]:
proxy.hv_output_enabled = True
proxy.frequency = 10e3

summary_df = pd.DataFrame()

plt.figure()
print 'voltage\t\tvgnd\t\t\tvrms\t\t\tvabs\t\tcapacitance'

proxy.select_on_board_test_capacitor(0)

for voltage in [50, 75, 100]:
    proxy.voltage = voltage
    time.sleep(.5)
    #sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads('A6', 200)
    sampling_rate, adc_settings, df_volts, df_adc_results = \
        proxy.analog_reads('PGA0', 100, gain_power=5)
    vgnd = np.mean(df_volts)[0]
    plt.plot(df_volts.values)
    vrms = np.sqrt(np.mean((df_volts - vgnd)**2))[0]
    vabs = np.mean(np.abs(df_volts - vgnd))[0]
    c = vabs / voltage * 0.15e-6
    print '%s\t\t%s\t%s\t%s\t%s' % (voltage, vgnd, vrms, vabs, c)
    summary_df = summary_df.append(pd.DataFrame(dict(voltage=[voltage], vgnd=[vgnd], vrms=[vrms], vabs=[vabs])),
                                   ignore_index=True)

proxy.hv_output_enabled = False

In [ ]:
p1 = plot_and_fit(summary_df['voltage'], summary_df['vrms'])
plt.xlabel('voltage')
plt.ylabel('vrms')
plt.title('y=%ex + %e' % (p1[0], p1[1]))

p1 = plot_and_fit(summary_df['voltage'], summary_df['vabs'])
plt.xlabel('voltage')
plt.ylabel('vabs')
plt.title('y=%ex + %e' % (p1[0], p1[1]))

p2 = plot_and_fit(summary_df['voltage'], summary_df['vgnd'])
plt.xlabel('voltage')
plt.ylabel('vgnd')
plt.title('y=%ex + %e' % (p2[0], p2[1]))

print p2[0] / p1[0]

In [ ]:
# test the programmable gain
proxy.frequency = 10e3
proxy.hv_output_enabled = True

plt.figure()
proxy.voltage = 100
for i in range(4):
    plt.figure()
    sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads('PGA0', 100, gain_power=i)
    plt.plot(df_adc_results.values)

In [ ]:
# measure the current on both sides of the boost converter
proxy.hv_output_enabled = True

voltage = np.linspace(proxy.min_waveform_voltage,
                      proxy.max_waveform_voltage,
                      5)[1:]

n_samples = 400
print "label\t\tv\t\trms"

summary_df = pd.DataFrame()
for label in ['A1', 'A2', 'A3']:
    for v in voltage:
        proxy.voltage = v
        time.sleep(.1)
        sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads(label, n_samples)
        summary_df = summary_df.append(pd.DataFrame(dict(voltage=v*np.ones(len(df_volts)),
                                                         label=[label]*len(df_volts),
                                                         mean=df_volts.values.flatten())),
                                       ignore_index=True)
        print "%s\t\t%.3f\t\t%.3f" % (label, v, np.sqrt(np.mean(df_volts**2))[0])

proxy.hv_output_enabled = False

In [ ]:
summary_df['rms'] = summary_df['mean']**2
for label, group in summary_df.groupby('label'):
    plt.figure()
    plt.plot(group['mean'])
    plt.title(label)
    plt.ylabel('Volts (RMS)')

plt.figure()
plt.plot(summary_df[summary_df['label']=='A1']['mean'] * 2e6 / 20e3 / 2.0)
plt.title('A1')
plt.ylabel('Volts (A)')
    
plt.figure()
plt.plot(summary_df[summary_df['label']=='A2']['mean'] / (51e3 / 5.1e3 * 1))
plt.title('A2')
plt.ylabel('Current (A)')

plt.figure()
plt.plot(summary_df[summary_df['label']=='A3']['mean'] / 0.03)
plt.title('A3')
plt.ylabel('Current (A)')

In [ ]:
"""
i = v_out / ( RB / RA * Rout )

RB = 51e3
RA = 5.1e3
Rout = 2.2
"""

plt.figure()
group = summary_df[summary_df['label']=='A2']
grouped = group.groupby('voltage', as_index=False).aggregate(np.mean)
grouped['rms'] = np.sqrt(grouped['rms'])

x = grouped['voltage']
y = grouped['rms'] / 51
p = plot_and_fit(x, y)
print p
plt.title('HV5702 Current')
plt.xlabel('Volts (RMS)')
plt.ylabel('Current (A)')


plt.figure()
group = summary_df[summary_df['label']=='A3']
grouped = group.groupby('voltage', as_index=False).aggregate(np.mean)
grouped['rms'] = np.sqrt(grouped['rms'])

x = grouped['voltage']
y = grouped['rms'] / 0.03
p = plot_and_fit(x, y)
print p
plt.title('MAX1771 Current')
plt.xlabel('Volts (RMS)')
plt.ylabel('Current (A)')

In [ ]:
proxy.voltage = 120
proxy.hv_output_enabled = True

n_samples = 400
print "label\t\ti\t\trms"

capacitance = np.array([0, 1e-12, 100e-12, 470e-12])

summary_df = pd.DataFrame()
for label in ['A1', 'A2', 'A3']:
    for i, c in enumerate(capacitance):
        proxy.select_on_board_test_capacitor(i)
        time.sleep(.5)
        sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads(label, n_samples)
        summary_df = summary_df.append(pd.DataFrame(dict(voltage=v*np.ones(len(df_volts)),
                                                         c=c*np.ones(len(df_volts)),
                                                         label=[label]*len(df_volts),
                                                         mean=df_volts.values.flatten())),
                                       ignore_index=True)
        print "%s\t\t%.3f\t\t%.3f" % (label, i, np.sqrt(np.mean(df_volts**2))[0])

summary_df['rms'] = summary_df['mean']**2
proxy.select_on_board_test_capacitor(-1)
proxy.hv_output_enabled = False

In [ ]:
group = summary_df[summary_df['label']=='A2']
grouped = group.groupby('c', as_index=False).aggregate(np.mean)
grouped['rms'] = np.sqrt(grouped['rms'])
x = grouped['c']
y = grouped['rms'] / 51
p = plot_and_fit(x, y)
print p
plt.title('HV5702 Current')
plt.xlabel('Capacitance (F)')
plt.ylabel('Current (A)')

plt.figure()
group = summary_df[summary_df['label']=='A3']
grouped = group.groupby('c', as_index=False).aggregate(np.mean)
grouped['rms'] = np.sqrt(grouped['rms'])

x = grouped['c']
y = grouped['rms'] / 0.03
p = plot_and_fit(x, y)
print p
plt.title('MAX1771 Current')
plt.xlabel('Capacitance (F)')
plt.ylabel('Current (A)')

In [ ]:
proxy.voltage = 100
proxy.hv_output_enabled = True

plt.figure()

def sample_feedback(i, plot=False, n_samples=100, gain_power=4):
    c = proxy.select_on_board_test_capacitor(i)
    #sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads('PGA0', n_samples, gain_power=gain_power)
    sampling_rate, adc_settings, df_volts, df_adc_results = proxy.analog_reads('A11', n_samples)
    v_gnd = np.mean(df_volts)
    v_rms = np.sqrt(np.mean((df_volts - v_gnd)**2))
    v_abs = np.abs(df_volts - v_gnd)
    v_abs_mean = np.mean(v_abs)
    filter_th = v_abs_mean * 1.1
    v_filtered_mean = np.mean(v_abs[v_abs < filter_th])
    c_meas = v_filtered_mean.values[0] / proxy.voltage * 0.15e-6
    
    if plot:
        plt.figure()
        plt.plot(df_volts.values - v_gnd[0])
        plt.plot(plt.xlim(), [filter_th, filter_th])
        plt.plot(plt.xlim(), [-filter_th, -filter_th])
        plt.figure()
        plt.hist(np.abs(df_volts.values - v_gnd[0]), 30)
        plt.plot([filter_th, filter_th], plt.ylim())
    return c_meas

c = []
for i in [-1, 0, 1, 2]:
    c = sample_feedback(i, True)
    print "ch %d: %.2f pF" % (i, c / 1e-12)

proxy.select_on_board_test_capacitor(-1)
proxy.hv_output_enabled = False